# Project Pre-Work

In [1]:
import acquire

from requests import get
from bs4 import BeautifulSoup
import os
import pandas as pd

## Explore github repo:
1.  Need to identify programing language
2.  Need to scrape text, and save as corpus

In [2]:
#https://github.com/microsoft/Web-Dev-For-Beginners
soup = acquire.make_soup('https://github.com/microsoft/Web-Dev-For-Beginners')

In [8]:
#[4] - Text
soup.select('div', class_ = "BorderGrid-cell")[5].text

'\n\n\n\n\n\n\n\n\n\n\n                    Why GitHub?\n                    \n\n\n\n\nFeatures →\n\nCode review\nProject management\nIntegrations\nActions\nPackages\nSecurity\nTeam management\nHosting\nMobile\n\n\nCustomer stories →\nSecurity →\n\n\n\n\n\nTeam\n\n\nEnterprise\n\n\n\n\n                    Explore\n                    \n\n\n\n\n\nExplore GitHub →\n\nLearn & contribute\n\nTopics\nCollections\nTrending\nLearning Lab\nOpen source guides\n\nConnect with others\n\nEvents\nCommunity forum\nGitHub Education\nGitHub Stars program\n\n\n\n\n\nMarketplace\n\n\n\n\n                    Pricing\n                    \n\n\n\n\nPlans →\n\nCompare plans\nContact Sales\n\n\nNonprofit →\nEducation →\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n        In this repository\n      \n\n        All GitHub\n      \n↵\n\n\n      Jump to\n      ↵\n\n\n\n\n\n\nNo suggested jump to results\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n        In this repository\n      \n\n        All GitHub\n      \n

In [4]:
soup.find('h2').text

'Latest commit'

In [5]:
<article class="markdown-body entry-content container-lg" itemprop="text"><h1><a id="user-content-web-development-for-beginners---a-curriculum" class="anchor" aria-hidden="true" href="#web-development-for-beginners---a-curriculum"><svg class="octicon octicon-link" viewBox="0 0 16 16" version="1.1" width="16" height="16" aria-hidden="true"><path fill-rule="evenodd" d="M7.775 3.275a.75.75 0 001.06 1.06l1.25-1.25a2 2 0 112.83 2.83l-2.5 2.5a2 2 0 01-2.83 0 .75.75 0 00-1.06 1.06 3.5 3.5 0 004.95 0l2.5-2.5a3.5 3.5 0 00-4.95-4.95l-1.25 1.25zm-4.69 9.64a2 2 0 010-2.83l2.5-2.5a2 2 0 012.83 0 .75.75 0 001.06-1.06 3.5 3.5 0 00-4.95 0l-2.5 2.5a3.5 3.5 0 004.95 4.95l1.25-1.25a.75.75 0 00-1.06-1.06l-1.25 1.25a2 2 0 01-2.83 0z"></path></svg></a>Web Development for Beginners - A Curriculum</h1>
<p>Azure Cloud Advocates at Microsoft are pleased to offer a 12-week, 24-lesson curriculum all about JavaScript, CSS, and HTML basics. Each lesson includes pre- and post-lesson quizzes, written instructions to complete the lesson, a solution, an assignment and more. Our project-based pedagogy allows you to learn while building, a proven way for new skills to 'stick'.</p>
<blockquote>
<p>Teachers, we have <a href="/microsoft/Web-Dev-For-Beginners/blob/main/for-teachers.md">included some suggestions</a> on how to use this curriculum. If you would like to create your own lessons, we have also included a <a href="/microsoft/Web-Dev-For-Beginners/blob/main/lesson-template/README.md">lesson template</a></p>
</blockquote>
<p><a href="https://youtube.com/watch?v=R1wrdtmBSII" title="Promo video" rel="nofollow"><img src="/microsoft/Web-Dev-For-Beginners/raw/main/screenshot.png" alt="Promo video" style="max-width:100%;"></a></p>
<blockquote>
<p>Click the image above for a video about the project and the folks who created it!</p>
</blockquote>
<h2><a id="user-content-pedagogy" class="anchor" aria-hidden="true" href="#pedagogy"><svg class="octicon octicon-link" viewBox="0 0 16 16" version="1.1" width="16" height="16" aria-hidden="true"><path fill-rule="evenodd" d="M7.775 3.275a.75.75 0 001.06 1.06l1.25-1.25a2 2 0 112.83 2.83l-2.5 2.5a2 2 0 01-2.83 0 .75.75 0 00-1.06 1.06 3.5 3.5 0 004.95 0l2.5-2.5a3.5 3.5 0 00-4.95-4.95l-1.25 1.25zm-4.69 9.64a2 2 0 010-2.83l2.5-2.5a2 2 0 012.83 0 .75.75 0 001.06-1.06 3.5 3.5 0 00-4.95 0l-2.5 2.5a3.5 3.5 0 004.95 4.95l1.25-1.25a.75.75 0 00-1.06-1.06l-1.25 1.25a2 2 0 01-2.83 0z"></path></svg></a>Pedagogy</h2>
<p>We have chosen two pedagogical tenets while building this curriculum: ensuring that it is project-based and that it includes frequent quizzes. By the end of this series, students will have built a typing game, a virtual terrarium, a 'green' browser extension, a 'space invaders' type game, and a business-type banking app, and will have learned the basics of JavaScript, HTML, and CSS along with the modern toolchain of today's web developer.</p>
<p>By ensuring that the content aligns with projects, the process is made more engaging for students and retention of concepts will be augmented. We also wrote several starter lessons in JavaScript basics to introduce concepts, paired with video from the "<a href="https://channel9.msdn.com/Series/Beginners-Series-to-JavaScript?WT.mc_id=cxaall-4621-cxall" rel="nofollow">Beginners Series to: JavaScript</a>" collection of video tutorials, some of whose authors contributed to this curriculum.</p>
<p>In addition, a low-stakes quiz before a class sets the intention of the student towards learning a topic, while a second quiz after class ensures further retention. This curriculum was designed to be flexible and fun and can be taken in whole or in part. The projects start small and become increasingly complex by the end of the 12 week cycle.</p>
<p>While we have purposefully avoided introducing JavaScript frameworks so as to concentrate on the basic skills needed as a web developer before adopting a framework, a good next step to completing this curriculum would be learning about Node.js via another collection of videos: "<a href="https://channel9.msdn.com/Series/Beginners-Series-to-Nodejs?WT.mc_id=cxaall-4621-cxall" rel="nofollow">Beginner Series to: Node.js</a>".</p>
<blockquote>
<p>Find our <a href="/microsoft/Web-Dev-For-Beginners/blob/main/CODE_OF_CONDUCT.md">Code of Conduct</a>, <a href="/microsoft/Web-Dev-For-Beginners/blob/main/CONTRIBUTING.md">Contributing</a>, and <a href="/microsoft/Web-Dev-For-Beginners/blob/main/TRANSLATIONS.md">Translation</a> guidelines. We welcome your constructive feedback!</p>
</blockquote>
<h2><a id="user-content-each-lesson-includes" class="anchor" aria-hidden="true" href="#each-lesson-includes"><svg class="octicon octicon-link" viewBox="0 0 16 16" version="1.1" width="16" height="16" aria-hidden="true"><path fill-rule="evenodd" d="M7.775 3.275a.75.75 0 001.06 1.06l1.25-1.25a2 2 0 112.83 2.83l-2.5 2.5a2 2 0 01-2.83 0 .75.75 0 00-1.06 1.06 3.5 3.5 0 004.95 0l2.5-2.5a3.5 3.5 0 00-4.95-4.95l-1.25 1.25zm-4.69 9.64a2 2 0 010-2.83l2.5-2.5a2 2 0 012.83 0 .75.75 0 001.06-1.06 3.5 3.5 0 00-4.95 0l-2.5 2.5a3.5 3.5 0 004.95 4.95l1.25-1.25a.75.75 0 00-1.06-1.06l-1.25 1.25a2 2 0 01-2.83 0z"></path></svg></a>Each lesson includes:</h2>
<ul>
<li>optional sketchnote</li>
<li>optional supplemental video</li>
<li>pre-lesson warmup quiz</li>
<li>written lesson</li>
<li>for project-based lessons, step-by-step guides on how to build the project</li>
<li>knowledge checks</li>
<li>a challenge</li>
<li>supplemental reading</li>
<li>assignment</li>
<li>post-lesson quiz</li>
</ul>
<h2><a id="user-content-lessons" class="anchor" aria-hidden="true" href="#lessons"><svg class="octicon octicon-link" viewBox="0 0 16 16" version="1.1" width="16" height="16" aria-hidden="true"><path fill-rule="evenodd" d="M7.775 3.275a.75.75 0 001.06 1.06l1.25-1.25a2 2 0 112.83 2.83l-2.5 2.5a2 2 0 01-2.83 0 .75.75 0 00-1.06 1.06 3.5 3.5 0 004.95 0l2.5-2.5a3.5 3.5 0 00-4.95-4.95l-1.25 1.25zm-4.69 9.64a2 2 0 010-2.83l2.5-2.5a2 2 0 012.83 0 .75.75 0 001.06-1.06 3.5 3.5 0 00-4.95 0l-2.5 2.5a3.5 3.5 0 004.95 4.95l1.25-1.25a.75.75 0 00-1.06-1.06l-1.25 1.25a2 2 0 01-2.83 0z"></path></svg></a>Lessons</h2>


SyntaxError: invalid syntax (<ipython-input-5-928bd9430340>, line 1)

In [ ]:
soup = acquire.make_soup('https://towardsdatascience.com/i-scraped-more-than-1k-top-machine-learning-github-profiles-and-this-is-what-i-found-1ab4fb0c0474')

In [ ]:
soup.text